Libraries

In [1]:
import json, os, pandas as pd, re, copy
cwd = os.getcwd()

Get the list of words that needs to be deleted before processing

In [11]:
remove_ingredient_before_fn = cwd + '/preprocess-now_youre_cooking_data/ingredient-lists/remove-ingredient-before.csv'
remove_ingredient_before_list = list(pd.read_csv(remove_ingredient_before_fn, header=None)[0])

Cleanup the recipes

In [4]:
recipe_json_fn = cwd + '/data/clean-recipes/recipes.json'
with open(recipe_json_fn, 'r') as fp:
    recipe_dict = json.load(fp)

In [5]:
# print(len(recipe_dict))
recipe_dict['0']

{u'categories': [u'cake/cookie', u'dessert', u'muffin'],
 u'ingredients': u'1 c  oil\t1 t  baking soda\t1 c  sugar\t1 t  salt\t1 c  brown sugar\t1 t  baking powder\t1 t  vanilla\t2 t  cinnamon\t4 x  eggs\t3 c  carrots; shredded\t2 c  flour, whole wheat\t1 c  walnuts; chopped\t1/3 c  dry milk',
 u'servings': u'12',
 u'steps': [u'in large bowl , blend oil and sugars on low until well mixed .',
  u'add vanilla .',
  u'beat in eggs , one at a time , blending well after each addition .',
  u'stir together dry ingredients and add to egg mixture until well blended .',
  u'stir in walnuts and carrots by hand .',
  u"pour batter into well greased and floured 10 '' tube pan or fluted pan .",
  u'bake at 350f for 50-60 minutes .',
  u'cool in pan , then top with powdered sugar or frosting of your choice .'],
 u'title': u'carrot cake'}

In [6]:
# clean_ingredients
def clean_ingredients(ingredient_items_str):
	# print(ingredient_items_str)
	# Remove the word 'Ingredients: ', any unicode, and strip newlines and spaces from both ends
	items = ingredient_items_str.strip()
	# items = ingredient_items_str.decode('unicode_escape').encode('ascii', 'ignore').strip()
	# Remove words listed in the remove-ingredient-before.csv using boundary search
	for each_ingredient in remove_ingredient_before_list:
		items = re.sub(r"\b" + re.escape(each_ingredient) + r"\b", '', items)
	# Replace the -LRB-, -RRB-, -LSB- and -RSB- with paranthases
	items = items.replace('-LRB-', '(').replace('-RRB-', ')')
	items = items.replace('-LSB-', '(').replace('-RSB-', ')')
	# Remove empty parenthases, and keep it as a bi-gram and replace the underscore with space to save them as uni-grams
	items = items.replace('(  )', '').replace('_', ' ')
	# Split the recipe into individual items (for both uni and bi-grams)
	item_arr = items.split('	')
	# # Get the number of items
	# num_items = len(item_arr)
	# # If the number of items in the recipe is less than the least_num_items, ignore this recipe and continue to the next one (however, increment the index)
	# if num_items < least_num_ingredients:
	# 	return None
	clean_ingredient_items_arr = []
	# Iterate over the items of the recipe
	for each_item in item_arr:
		# If the item ends with semicolumn then exclude it, and no need to keep track of its index
		if each_item.endswith(':'):
			continue
		# If the item has the following conditions (completion of prev item), then exclude it, and no need to keep track of its index
		if each_item.startswith('(') or each_item.startswith('-') or (each_item.endswith(')') and not('(' in each_item)):
			continue
		# Preserve the item before the next cleaning part, so we can save it if it's a strange item
		each_item_b4_strip = each_item
		# Strip spaces and commas from both ends, and extra spaces in the middle
		each_item = each_item.strip(' ').strip(',').strip(' ').strip(',')
		each_item = re.sub(r' +', ' ', each_item)
		if not re.search('[a-zA-Z]', each_item):
			strange_items_list.append(each_item_b4_strip)
			continue
		# Append the items to the 
		clean_ingredient_items_arr.append(each_item)
	# # Get the number of clean items
	# num_clean_items = len(clean_ingredient_items_arr)
	# # If the number of clean items in the recipe is less than the least_num_items,
	# # ignore this recipe and continue to the next one (however, increment the index)
	# if num_clean_items < least_num_ingredients:
	# 	return None
	return clean_ingredient_items_arr
# end clean_ingredients

Cleanup the ingredients

In [14]:
# Set least number of ingredients that should be in a recipe
least_num_ingredients = 2
# Set the list of strange items
strange_items_list = []
# Copy to a new dict which would store the cleaned ingredients
clean_recipe_dict = copy.deepcopy(recipe_dict)
# Iterate over the recipe dictionary
for each_recipe in clean_recipe_dict:
	if 'ingredients' in clean_recipe_dict[each_recipe]:
		clean_recipe_dict[each_recipe]['ingredients'] = clean_ingredients(clean_recipe_dict[each_recipe]['ingredients'])
# print('Number of strange items:', len(strange_items_list))
print('Clean recipes dict:', len(clean_recipe_dict))
print('Number of strange items:', len(strange_items_list))

Indexing for ingredient-phrase-tagger tool